## Trabalhando com Big Data - 25 Milhões de Registros
fonte: https://grouplens.org/datasets/movielens/

In [1]:
import pandas as pd

In [2]:
!pwd

'pwd' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [3]:
!ls -ilah /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m

'ls' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [4]:
!head /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/movies.csv

'head' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [5]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'ratings.csv'
url

'/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/ratings.csv'

In [6]:
%time

dfratings = pd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfratings)))

Wall time: 0 ns


FileNotFoundError: [Errno 2] No such file or directory: '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/ratings.csv'

In [ ]:
dfratings.head()

In [ ]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'movies.csv'
url

In [ ]:
%time

dfmovies = pd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfmovies)))

In [ ]:
dfmovies.head()

In [ ]:
%%time

dfdrama_acao = dfmovies[ dfmovies.genres.str.contains('Drama') |  dfmovies.genres.str.contains('Action') ]
print('total de registros: {:,d}'.format(len(dfdrama_acao)))
dfdrama_acao.head()

# Obter todos os generos

In [ ]:
genres = 'Comedy|Drama|Romance'
generos = genres.split('|')
generos

In [ ]:
%%time

set_generos = set()

for index, row in dfmovies.iterrows(): 
    generos = row['genres'].split('|')
    set_generos.update(generos)
    
generos = list(set_generos)
print('numero de generos',len(generos))
print(generos)

## Criando colunas Genero - individuais

In [ ]:
%%time

for genero in generos:
    dfmovies[genero] = False
    
dfmovies.head()

In [ ]:
%%time 

for index, row in dfmovies.iterrows(): 
    generos = row['genres'].split('|')
    for genero in generos:
        dfmovies.at[index, genero] = True
    
dfmovies.head()

In [ ]:
dfmovies.Drama.value_counts()

In [ ]:
del dfmovies['genres']
dfmovies.head()

# criar estatistica de generos

In [ ]:
generos = list(set_generos)
print('numero de generos',len(generos))
print(generos)

In [ ]:
dfmovies.Horror.value_counts()

In [ ]:
for genero in generos:
    count_genero = dfmovies[genero].sum()
    print(genero, count_genero)
    break

In [ ]:
dict_genero = {}
for genero in generos:
    count_genero = dfmovies[genero].sum()
    dict_genero[genero] = count_genero

dict_genero

In [ ]:
df_genero_cont = pd.DataFrame.from_dict(dict_genero, orient='index').reset_index()
df_genero_cont

In [ ]:
df_genero_cont.columns = ['genero', 'total']
df_genero_cont

In [ ]:
dfo = df_genero_cont.sort_values(by='total', ascending=False).reset_index(drop=True)
dfo

# Capturando o ano

In [ ]:
dfmovies.head()

In [ ]:
x = 'Father of the Bride Part II (1995)'
x.find(')')

In [ ]:
type(x)

In [ ]:
x = 'Father of the Bride Part II'
x.find(')')

In [ ]:
import numpy as np

dfmovies['ano'] = dfmovies.title.apply(lambda x : x.split('(')[1] if x.find(')') != -1  else np.nan)
dfmovies.head()

In [ ]:
dfmovies['ano'] = dfmovies.ano.apply(lambda x : x.replace(')', '') if type(x) == str else x)
dfmovies.head()

In [ ]:
print(dfmovies.ano.value_counts())

In [ ]:
import re

x1 = '19993'
x2 = '1 2 title'
datas = r'^([1-9][0-9][0-9][0-9])'
#re.match(r'\d{4}', x)

if re.match(datas, x1):
    print('ok')
if re.match(datas, x2):
    print('ok nao data')

In [ ]:
datas = r'^([0-9][0-9][0-9][0-9])'

dfmovies['ano'] = dfmovies.title.apply(lambda x : x.split('(')[1] if x.find(')') != -1  else np.nan)
dfmovies['ano'] = dfmovies.ano.apply(lambda x : x.replace(')', '') if type(x) == str else x)

dfmovies['ano'] = dfmovies.ano.apply(lambda x : x[:4] if x is not np.nan and re.match(datas, x[:4]) else np.nan)
dfmovies.head()

In [ ]:
dfmovies.ano.value_counts()

In [ ]:
len(dfmovies.ano.unique())

In [ ]:
print(dfmovies.ano.unique())

In [ ]:
dfmovies.ano.isnull().sum(), len(dfmovies.ano)

In [ ]:
dfanofilmes = dfmovies.ano.value_counts().to_frame().reset_index()
dfanofilmes.columns = ['ano', 'total_filmes']
dfanofilmes.head()

In [ ]:
dfanofilmes = dfanofilmes.sort_values(by='total_filmes', ascending=False).reset_index(drop=True)
dfanofilmes.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,8))

sns.barplot(x='total_filmes', y='ano', data=dfanofilmes[:20], palette='Greens_r')

# Join - inner join

In [ ]:
dfratings.head()

In [ ]:
dfmovies.head()

In [ ]:
%%time

dffilmes =  pd.merge(dfmovies, dfratings, left_on='movieId', right_on='movieId', how="inner")
print('total de registros: {:,d}'.format(len(dffilmes)))
dffilmes.head()

In [ ]:
dffilmes.info()

In [ ]:
dffilmes.sort_values(by='rating', ascending=False)[:15]

In [ ]:
# generos = ['Crime', 'Western', 'Thriller', 'War', 'Sci-Fi', 'Children', 
#             'Documentary', '(no genres listed)', 'Romance', 'Adventure', 'Mystery',
#             'Comedy', 'IMAX', 'Animation', 'Action', 'Horror', 'Film-Noir', 'Fantasy',
#         'Drama', 'Musical'] 

In [ ]:
%%time

#dffilmes.to_csv('filmes-2022.csv', index=False)
#!ls -ilah *.csv

In [ ]:
!rm filmes-2022.csv
!ls -ilah *.csv

In [ ]:
dffilmes.Action.value_counts()

In [ ]:
acao = dffilmes.Action.value_counts()[1]
acao

In [ ]:
dffilmes.Drama.value_counts()

In [ ]:
drama = dffilmes.Drama.value_counts()[1]
drama

In [ ]:
dffilmes.Adventure.value_counts()

In [ ]:
aventura = dffilmes.Adventure.value_counts()[1]
aventura

In [ ]:
dffilmes.Romance.value_counts()

In [ ]:
romance = dffilmes.Romance.value_counts()[1]
romance

In [ ]:
#outros = len(dfmovies) - aventura - drama - romance - acao
#outros

# grafico de Pizza - genero

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

plt.rcParams["figure.figsize"] = (9,9)

#define data
data =    [aventura, drama, romance, acao] #f6.percent
labels = ['aventura', 'drama', 'romance', 'acao'] #, 'outros'] #df.rating

#define Seaborn color palette to use
colors = sns.color_palette('pastel')#[0:4]

colors = sns.color_palette("rocket")

colors = sns.color_palette("Blues_r") #, as_cmap=True)

#colors = 'Blues'



fig = plt.figure()

#ax = fig.add_axes([.8, .8, .8, .8], aspect=2)

ax = fig.add_axes([.8, .8, .8, .8], aspect=2)
ax.set_title('Filmes por genero ')
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')


plt.show()

In [ ]:
dfo = dffilmes.rating.value_counts()
dfo

In [ ]:
type(dfo)

In [ ]:
df2 = dfo.to_frame().reset_index()
df2.columns = ['rating', 'quantidade']

In [ ]:
df2

In [ ]:
df2['percent'] = df2['quantidade'] / df2['quantidade'].sum() * 100
df2

In [ ]:
# formatar a saída

df3 = df2.style.format({
    "rating": "{:.1f}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})



df3

In [ ]:
df4 = df2[ df2.percent >= 7]
df4

In [ ]:
df4.percent.sum()

In [ ]:
print('{:,d}'.format(df4.quantidade.sum()))

In [ ]:
df5 = df2[ df2.percent < 7]
df5

In [ ]:
print('{:,d}'.format(df5.quantidade.sum()))

In [ ]:
df5.percent.sum()

In [ ]:
df5

In [ ]:
row = {'rating': 'outros', 'quantidade': df5.quantidade.sum(), 'percent': df5.percent.sum()  }
row

In [ ]:
df6 = df4.append(row, ignore_index=True)
df6

In [ ]:
# formatar a saída

dfs = df6.style.format({
    "rating": "{}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})

dfs

In [ ]:
df6.to_csv('ratings_saida.csv', index=False)

In [ ]:
!cat ratings_saida.csv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (9,9)

#define data
data = df6.percent
labels = df6.rating

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:7]

colors = sns.color_palette("Blues_r")

colors = sns.color_palette("Reds_r")[0:7]

colors = sns.color_palette("Blues_r")[0:7]


colors = sns.color_palette("Oranges")[0:10]


colors = sns.color_palette("Blues_r")

fig = plt.figure()

ax = fig.add_axes([0, 0, .8, .8], aspect=1)

ax.set_title('Filmes por avaliação (1-5)')
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')


plt.show()

In [ ]:
!ls -ilah /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m

In [ ]:
!head /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/tags.csv

# mapa

In [ ]:
!pip install plotly==5.5.0

# covid19

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/02-04-2022.csv'
url


In [ ]:
import pandas as pd

df = pd.read_csv(url)
df.head()

In [ ]:
df.info()

In [ ]:
df.loc[0, 'Province_State']

In [ ]:
df2 = df.groupby('Country_Region').sum().reset_index()
df2.head(30)

In [ ]:
len(df2)

In [ ]:
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
df3codes = pd.read_csv(url)
df3codes.head()

In [ ]:
dfmap = pd.merge(df2, df3codes, left_on='Country_Region', right_on='name', how='inner')
print(len(dfmap))
dfmap.head()

In [ ]:
dfmap.columns

In [ ]:
dfmap.Country_Region.unique()

In [ ]:
dfmap.Country_Region.value_counts()

In [ ]:
import plotly.graph_objects as go
import pandas as pd


fig = go.Figure(data=go.Choropleth(
    locations = dfmap['alpha-3'],
    z = dfmap['Confirmed'],
    text = dfmap['Country_Region'],
    colorscale = 'Reds',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Casos Confirmados',
))

fig.update_layout(
    title_text='Covid19 - 4/fevereiro/2022 - Casos Confirmados',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [
    ]
)

fig.show()

In [ ]:
dfmap.head()

In [ ]:
dfmap.to_csv('mapa_mundo_covid19-4-fevereiro-2022.csv')
!ls -ilah m*.csv

In [ ]:
!pwd

# Séries Teporais - Pandas

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url

In [ ]:
import pandas as pd

df = pd.read_csv(url)
print(len(df))
df.head()

In [ ]:
df2 = df.T
df2.head()

In [ ]:
datas_s = list(df.columns[4:])
print(len(datas_s))
datas_s[:5]

In [ ]:
# convertar a string para o formato data

datas = pd.to_datetime(datas_s)
datas[:10]

In [ ]:
datas[:-5]

In [ ]:
df.head()

## convertendo as string para DATAS

In [ ]:
datas_str = list(df.columns[4:])
datas_str[:10]

In [ ]:
df2.head()

In [ ]:
cols = list(df2.columns[4:])
cols[:3]

In [ ]:
datas2 = list(datas)
datas2[:3]

In [ ]:
from datetime import datetime

df2 = df.copy()

for col, data in zip(list(df.columns[4:]), datas2):
    #print(col, data)
    date_time = datetime.strftime(data, "%Y-%m-%d")
    #print(date_time)
    df2.rename(columns={ col: date_time}, inplace = True)

df2.head()

In [ ]:
df2.head()

# somar os países com métricas dos estados

In [ ]:
# somar os países com métricas dos estados
print(len(df2))
dfm = df2.groupby('Country/Region').sum().reset_index()
print(len(dfm))
dfm.head(30)

In [ ]:
br = dfm [ dfm['Country/Region'].str.contains("Brazil") ]
br

In [ ]:
x = br.columns[4:]
x

In [ ]:
y = br[x].values
y = y.ravel()
y

In [ ]:
br.T

In [ ]:
br2 = br.T[4:]
br2.rename(columns={ 23: 'Confirmados'}, inplace = True)
br2

In [ ]:
import plotly.express as px

df = br2.copy()

# plotly
fig = px.line(df, x=df.index, y = df.columns)
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
dfm.head()

In [ ]:
dfm [dfm['Country/Region'].str.contains("US") ]

In [ ]:
top = dfm [ dfm['Country/Region'].str.contains("Brazil") |
          dfm['Country/Region'].str.contains("USA") |
          dfm['Country/Region'].str.contains("India") |
          dfm['Country/Region'].str.contains("Russia") |
          dfm['Country/Region'].str.contains("US")
         ]
top

In [ ]:
top2 = top.T[4:]
top2

In [ ]:
top2.rename(columns={ 23: 'Brasil', 79: 'India', 144: 'Russia', 182: "US"}, inplace = True)
top2

In [ ]:
# plotly
df = top2.copy()

fig = px.line(df, x=df.index, y = df.columns)
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
top2.head(8)

In [ ]:
top2['Casos_Diarios_US'] = top2['US'] - top2.shift(1)['US']
top2.head(8)

In [ ]:
top2['Casos_Diarios_Brasil'] = top2['Brasil'] - top2.shift(1)['Brasil']
top2.head()

In [ ]:
top2.tail()

In [ ]:
# plotly
df = top2.copy()

y = ['Casos_Diarios_US', 'Casos_Diarios_Brasil']
fig = px.line(df, x=df.index, y = y)
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
df = top2.copy()

y =  ['Casos_Diarios_Brasil']
fig = px.line(df, x=df.index, y = y)
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
print(len(df))
df = df[df['Casos_Diarios_Brasil'] >= 0] 
print(len(df))

In [ ]:

y =  ['Casos_Diarios_Brasil']
fig = px.line(df, x=df.index, y = y)
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
# Using plotly.express
import plotly.express as px

df = px.data.stocks()
fig = px.line(df, x='date', y="GOOG")
fig.show()

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df = px.data.stocks()

In [ ]:
print(len(df))
df = top2.copy()

df = df.dropna()

print(len(df))

In [ ]:
import numpy as np

print(len(df))

df = top2.copy()

df 

df = df[df['Casos_Diarios_Brasil'] >= 0] 

print(len(df))

In [ ]:
import plotly.express as px


#df = px.data.stocks(indexed=True)-1
y =  "Casos_Diarios_Brasil"
fig = px.bar(df, x=df.index, y=y)

fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
df.tail()

In [ ]:
dfo = df.sort_values(by='Casos_Diarios_Brasil', ascending=False)
dfo.head(8)

In [ ]:
y =  dfo['Casos_Diarios_Brasil']
fig = px.bar(dfo, x=dfo.index[:30], y=y[:30])

fig.update_layout(template="plotly_dark")
fig.show()